In [34]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img
from keras.models import Sequential
import numpy as np
from glob import glob
from matplotlib import pyplot as plt

from keras.models import load_model
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tf_explain.core.smoothgrad import SmoothGrad
from keras.utils import img_to_array, load_img
import cv2

from PIL import Image, ImageDraw, ImageFont
import PIL.Image
from matplotlib import pylab as P
import saliency.core as saliency


import tensorflow as tf

In [35]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('../../Covid19-dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 251 images belonging to 3 classes.


In [10]:
test_set = test_datagen.flow_from_directory('../../Covid19-dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [11]:
model = load_model('../../models/covid/covid_vgg1.h5')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [6]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [43]:
# Boilerplate methods.
def ShowImage(im, title='', ax=None):
  if ax is None:
    P.figure()
  P.axis('off')
  P.imshow(im)
  P.title(title)

def ShowGrayscaleImage(im, title='', ax=None):
  if ax is None:
    P.figure()
  P.axis('off')

  P.imshow(im, cmap=P.cm.gray, vmin=0, vmax=1)
  P.title(title)

def ShowHeatMap(im, title, ax=None):
  if ax is None:
    P.figure()
  P.axis('off')
  P.imshow(im, cmap='inferno')
  P.title(title)

def LoadImage(file_path):
  im = PIL.Image.open(file_path)
  im = im.resize((224,224))
  im = np.asarray(im)
  return im

def PreprocessImage(im):
  im = tf.keras.applications.vgg16.preprocess_input(im)
  return im

In [44]:
def call_model_function(im, call_model_args=None, expected_keys=None):
    # Get the logits tensor from the model
    logits_tensor = model.output

    # Get the input tensor
    input_tensor = model.input

    # Calculate gradients
    with tf.GradientTape() as tape:
        tape.watch(input_tensor)
        logits = logits_tensor[0, call_model_args['prediction_class']]
    gradients = tape.gradient(logits, input_tensor)
    return gradients.numpy()


In [75]:
def visualize_image_grayscale(image_3d):
    # Convert the 3D tensor to 2D grayscale
    grayscale_image = np.mean(image_3d, axis=-1)
    # Rescale the values to [0, 1]
    grayscale_image -= np.min(grayscale_image)
    grayscale_image /= np.max(grayscale_image)
    # Convert the values to the range [0, 255] and cast to uint8
    grayscale_image = (grayscale_image * 255).astype(np.uint8)
    # Resize the image to 224x224
    grayscale_image_resized = cv2.resize(grayscale_image, (224, 224), interpolation=cv2.INTER_LINEAR)
    return grayscale_image_resized

In [76]:
def generate_saliency_maps(image_path, model, class_idx_str):
    # Load and preprocess the image
    im = load_img(image_path, target_size=(224, 224))
    im = img_to_array(im)
    im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    im = np.expand_dims(im, axis=0)

    print("Image loaded and preprocessed.")

    # Get the index of the target class
    prediction_class = int(class_idx_str)

    # Define a function to compute the gradients
    def compute_gradients(x_value_batch, call_model_args=None, expected_keys=None):
        x_value = x_value_batch[0]  # Unbatch the input
        x_value = tf.convert_to_tensor(x_value)  # Convert the input to a TensorFlow tensor
        with tf.GradientTape() as tape:
            tape.watch(x_value)
            output = model(x_value)
            class_output = output[:, prediction_class]

        gradients = tape.gradient(class_output, x_value)
        gradients_batched = np.expand_dims(gradients, axis=0)  # Add the batch dimension
        return {saliency.INPUT_OUTPUT_GRADIENTS: gradients_batched}

    print("Gradient computation function defined.")

    gradient_saliency = saliency.GradientSaliency()

    # Compute the vanilla mask and the smoothed mask.
    vanilla_mask_3d = gradient_saliency.GetMask(im[0], compute_gradients)
    smoothgrad_mask_3d = gradient_saliency.GetSmoothedMask(im[0], compute_gradients)

    print("Saliency masks computed.")

    # Call the visualization methods to convert the 3D tensors to 2D grayscale.
    vanilla_mask_grayscale = visualize_image_grayscale(vanilla_mask_3d)
    smoothgrad_mask_grayscale = visualize_image_grayscale(smoothgrad_mask_3d)

    print("Saliency masks converted to grayscale.")

    # Save the results as image files
    cv2.imwrite('vanilla_mask_grayscale.png', vanilla_mask_grayscale)
    cv2.imwrite('smoothgrad_mask_grayscale.png', smoothgrad_mask_grayscale)

    print("Saliency maps saved as 'vanilla_mask_grayscale.png' and 'smoothgrad_mask_grayscale.png'.")


# Call the generate_saliency_maps function with an image path and your model
generate_saliency_maps('../../Covid19-dataset/test/Covid/0100.jpeg', model, '2')


Image loaded and preprocessed.
Gradient computation function defined.
Saliency masks computed.
Saliency masks converted to grayscale.


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:699: error: (-215:Assertion failed) image.channels() == 1 || image.channels() == 3 || image.channels() == 4 in function 'cv::imwrite_'


In [78]:
def visualize_image_grayscale(image_3d):
    # Convert the 3D tensor to 2D grayscale
    grayscale_image = np.mean(image_3d, axis=-1)
    # Rescale the values to [0, 1]
    grayscale_image -= np.min(grayscale_image)
    grayscale_image /= np.max(grayscale_image)
    # Convert the values to the range [0, 255] and cast to uint8
    grayscale_image = (grayscale_image * 255).astype(np.uint8)
    # Resize the image to 224x224
    grayscale_image_resized = cv2.resize(grayscale_image, (224, 224), interpolation=cv2.INTER_LINEAR)
    return grayscale_image_resized


def generate_saliency_maps(image_path, model, class_idx_str):
    # Load and preprocess the image
    im = load_img(image_path, target_size=(224, 224))
    im = img_to_array(im)
    im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    im = np.expand_dims(im, axis=0)

    print("Image loaded and preprocessed.")

    # Get the index of the target class
    prediction_class = int(class_idx_str)

    # Define a function to compute the gradients
    def compute_gradients(x_value_batch, call_model_args=None, expected_keys=None):
        x_value = x_value_batch[0]  # Unbatch the input
        x_value = tf.convert_to_tensor(x_value)  # Convert the input to a TensorFlow tensor
        with tf.GradientTape() as tape:
            tape.watch(x_value)
            output = model(x_value)
            class_output = output[:, prediction_class]

        gradients = tape.gradient(class_output, x_value)
        gradients_batched = np.expand_dims(gradients, axis=0)  # Add the batch dimension
        return {saliency.INPUT_OUTPUT_GRADIENTS: gradients_batched}

    print("Gradient computation function defined.")

    gradient_saliency = saliency.GradientSaliency()

    # Compute the vanilla mask and the smoothed mask.
    vanilla_mask_3d = gradient_saliency.GetMask(im[0], compute_gradients)
    smoothgrad_mask_3d = gradient_saliency.GetSmoothedMask(im[0], compute_gradients)

    print("Saliency masks computed.")

    # Call the visualization methods to convert the 3D tensors to 2D grayscale.
    vanilla_mask_grayscale = visualize_image_grayscale(vanilla_mask_3d)
    smoothgrad_mask_grayscale = visualize_image_grayscale(smoothgrad_mask_3d)

    print("Saliency masks converted to grayscale.")

    # Save the results as image files
    cv2.imwrite('vanilla_mask_grayscale.png', vanilla_mask_grayscale)
    cv2.imwrite('smoothgrad_mask_grayscale.png', smoothgrad_mask_grayscale)

    print("Saliency maps saved as 'vanilla_mask_grayscale.png' and 'smoothgrad_mask_grayscale.png'.")

    # Print out the input image and the saliency maps
    input_image = cv2.imread(image_path)
    input_image_resized = cv2.resize(input_image, (224, 224), interpolation=cv2.INTER_LINEAR)
    cv2.imshow("Input Image", input_image_resized)
    cv2.imshow("Vanilla Saliency Map", vanilla_mask_grayscale)
    cv2.imshow("SmoothGrad Saliency Map", smoothgrad_mask_grayscale)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Call the generate_saliency_maps function with an image path and your model
generate_saliency_maps('../../Covid19-dataset/test/Covid/0100.jpeg', model, '2')


Image loaded and preprocessed.
Gradient computation function defined.
Saliency masks computed.
Saliency masks converted to grayscale.
Saliency maps saved as 'vanilla_mask_grayscale.png' and 'smoothgrad_mask_grayscale.png'.
